In [2]:
import random

"""
Course: Computer Science
Lab 10: Simulate the Diffie–Hellman key exchange
Goal: Demonstrate how a Man-in-the-Middle attack can intercept and manipulate shared keys.

Requirements:
1. Implement standard Diffie–Hellman key exchange using a prime number and generator.
2. Create three entities: Alice, Bob, and Tom (the attacker).
3. Make Tom intercept and replace the public keys exchanged between Alice and Bob.
4. Show that:
    - Alice shares a secret key with Tom
    - Bob shares a different secret key with Tom
    - Alice and Bob do NOT share the same key
5. Print all keys clearly to demonstrate the attack.
"""

class DH_Endpoint:
    """Represents an honest participant (Alice or Bob)."""
    def __init__(self, name, p, g):
        self.name = name
        self.p = p
        self.g = g
        # 1. Generate Private Key (Random Integer)
        self.private_key = random.randint(1, p - 2)
        # 2. Calculate Public Key: g^private_key mod p
        self.public_key = pow(g, self.private_key, p)
        self.shared_secret = None

    def generate_shared_secret(self, received_public_key):
        """Calculates the shared secret using the received public key."""
        # Secret = received_key^private_key mod p
        self.shared_secret = pow(received_public_key, self.private_key, self.p)
        return self.shared_secret

class DH_Attacker:
    """Represents the Man-in-the-Middle (Tom)."""
    def __init__(self, name, p, g):
        self.name = name
        self.p = p
        self.g = g
        # Tom generates his own private key to fool both parties
        self.private_key = random.randint(1, p - 2)
        self.public_key = pow(g, self.private_key, p)
        
        # Tom will have two separate secrets: one shared with Alice, one with Bob
        self.secret_with_alice = None
        self.secret_with_bob = None

    def intercept_alice_and_calculate(self, alice_public_key):
        """Tom uses Alice's public key to calculate a shared secret with her."""
        self.secret_with_alice = pow(alice_public_key, self.private_key, self.p)
    
    def intercept_bob_and_calculate(self, bob_public_key):
        """Tom uses Bob's public key to calculate a shared secret with him."""
        self.secret_with_bob = pow(bob_public_key, self.private_key, self.p)

def run_simulation():
    print("===========================================================")
    print("       LAB 10: DIFFIE-HELLMAN MITM SIMULATION")
    print("===========================================================\n")

    # 1. PUBLIC PARAMETERS
    # Using small numbers for demonstration clarity. 
    p = 23719  # Prime number
    g = 11     # Generator
    print(f"[Step 0] Global Parameters:\n  Prime (p): {p}\n  Generator (g): {g}\n")

    # 2. INITIALIZE ENTITIES
    alice = DH_Endpoint("Alice", p, g)
    bob = DH_Endpoint("Bob", p, g)
    tom = DH_Attacker("Tom", p, g)

    print("[Step 1] Key Generation Phase (Private & Public Keys):")
    print(f"  Alice | Private: {alice.private_key:<5} | Public: {alice.public_key}")
    print(f"  Bob   | Private: {bob.private_key:<5} | Public: {bob.public_key}")
    print(f"  Tom   | Private: {tom.private_key:<5} | Public: {tom.public_key} (Attacker)")
    print("-" * 60)

    # 3. THE ATTACK (INTERCEPTION)
    print("[Step 2] Exchange Phase (Tom Intercepts & Replaces Keys):")
    
    # Alice tries to send her Public Key to Bob, but Tom intercepts it.
    print(f"  A. Alice sends Public Key ({alice.public_key}) -> Intended for Bob")
    print(f"     [INTERCEPTION] Tom grabs Alice's key.")
    print(f"     [REPLACEMENT]  Tom sends HIS Public Key ({tom.public_key}) -> To Bob")
    tom.intercept_alice_and_calculate(alice.public_key)
    
    # Bob receives Tom's key (thinking it is Alice's)
    bob_received_key = tom.public_key 
    print("")
    
    # Bob tries to send his Public Key to Alice, but Tom intercepts it.
    print(f"  B. Bob sends Public Key ({bob.public_key}) -> Intended for Alice")
    print(f"     [INTERCEPTION] Tom grabs Bob's key.")
    print(f"     [REPLACEMENT]  Tom sends HIS Public Key ({tom.public_key}) -> To Alice")
    tom.intercept_bob_and_calculate(bob.public_key)

    # Alice receives Tom's key (thinking it is Bob's)
    alice_received_key = tom.public_key

    print("-" * 60)

    # 4. SECRET CALCULATION
    print("[Step 3] Secret Derivation Phase:")
    
    # Alice calculates secret using what she thinks is Bob's key (actually Tom's)
    alice_secret = alice.generate_shared_secret(alice_received_key)
    
    # Bob calculates secret using what he thinks is Alice's key (actually Tom's)
    bob_secret = bob.generate_shared_secret(bob_received_key)

    print(f"  Alice's Calculated Secret: {alice_secret}")
    print(f"  Bob's Calculated Secret:   {bob_secret}")
    print(f"  Tom's Secret with Alice:   {tom.secret_with_alice}")
    print(f"  Tom's Secret with Bob:     {tom.secret_with_bob}")
    print("-" * 60)

    # 5. VERIFICATION OF ATTACK (Matching Lab Requirements)
    print("[Step 4] Final Verification of Attack Success:")
    
    # Requirement: Alice shares a secret key with Tom
    print("\n1. Does Alice share a secret with Tom?")
    print(f"   Alice: {alice_secret} vs Tom: {tom.secret_with_alice}")
    if alice_secret == tom.secret_with_alice:
        print("   -> YES [Requirement Met]")
    else:
        print("   -> NO")

    # Requirement: Bob shares a different secret key with Tom
    print("\n2. Does Bob share a secret with Tom?")
    print(f"   Bob:   {bob_secret} vs Tom: {tom.secret_with_bob}")
    if bob_secret == tom.secret_with_bob:
        print("   -> YES [Requirement Met]")
    else:
        print("   -> NO")

    # Requirement: Alice and Bob do NOT share the same key
    print("\n3. Do Alice and Bob share the same key?")
    print(f"   Alice: {alice_secret} vs Bob: {bob_secret}")
    if alice_secret != bob_secret:
        print("   -> NO  [Requirement Met: Communication is compromised]")
    else:
        print("   -> YES [Attack Failed]")

if __name__ == "__main__":
    run_simulation()

       LAB 10: DIFFIE-HELLMAN MITM SIMULATION

[Step 0] Global Parameters:
  Prime (p): 23719
  Generator (g): 11

[Step 1] Key Generation Phase (Private & Public Keys):
  Alice | Private: 20007 | Public: 4886
  Bob   | Private: 4141  | Public: 20383
  Tom   | Private: 22415 | Public: 18674 (Attacker)
------------------------------------------------------------
[Step 2] Exchange Phase (Tom Intercepts & Replaces Keys):
  A. Alice sends Public Key (4886) -> Intended for Bob
     [INTERCEPTION] Tom grabs Alice's key.
     [REPLACEMENT]  Tom sends HIS Public Key (18674) -> To Bob

  B. Bob sends Public Key (20383) -> Intended for Alice
     [INTERCEPTION] Tom grabs Bob's key.
     [REPLACEMENT]  Tom sends HIS Public Key (18674) -> To Alice
------------------------------------------------------------
[Step 3] Secret Derivation Phase:
  Alice's Calculated Secret: 10863
  Bob's Calculated Secret:   492
  Tom's Secret with Alice:   10863
  Tom's Secret with Bob:     492
-----------------------